<h2>Introduction</h2>

Spotify is one of the leading music streaming platforms in the world today. Founded in 2006, they initially used pirated tracks to demonstrate their proof of concept, but has since worked with record companies, big and small, and even with independent artists to rack up billions of streams across the world daily, thanks to their massive user base and proprietary algorithms.

<h2>Building the database</h2>

Using Python as our primary tool, we will build a database of Daily Top 200 Tracks, with a selection of regions and a specifiable date range. This can be accomplished through web scraping using Selenium Webdriver to simulate logins and opening of the chart page for each date, reading the HTML elements of that page, inserting into a SQLite database, then moving onto the chart page for the next date.

In [ ]:
import pandas as pd

In [ ]:
# regions in South East Asia with Spotify availability
# the regions can be split across multiple scraping notebooks and run simulatenously to save time, but keep in mind request limits
region_dict = {
    "id" : "Indonesia",
    "my" : "Malaysia",
    "ph" : "Philippines",
    "sg" : "Singapore",
    "th" : "Thailand",
    "vn" : "Vietnam"
}

In [ ]:
# specify date range
start_date = '2017-01-01'
end_date = '2023-08-28'
dates_list = pd.date_range(start=start_date, end=end_date).sort_values(ascending=True)

In [ ]:
# the scraper uses the firefox engine Geckodriver and sqlite as the database, make sure to have it installed
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import sqlite3

In [ ]:
options = webdriver.FirefoxOptions()

driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=options)
print("Driver start")

driver.get("https://accounts.spotify.com/en/login?continue=https%3A%2F%2Fcharts.spotify.com/login")

username_field = driver.find_element(by="id", value="login-username")  # Replace with the actual username field ID
password_field = driver.find_element(by="id", value="login-password")  # Replace with the actual password field ID
login_button = driver.find_element(by="id", value="login-button")  # Replace with the actual login button ID

# Enter login credentials
username_field.send_keys("spotify-username")
password_field.send_keys("spotify-password")

# Click the login button
login_button.click()

# Now you can perform your automation tasks after logging in

wait = WebDriverWait(driver, 10)
try:
    wait.until(EC.url_to_be('https://charts.spotify.com/charts/overview/global'))
    print("login success, redirecting")
    print("***")
    
    conn = sqlite3.connect('data/charts-sea.db')
    cursor = conn.cursor()
    
    # main scraping logic
    for key in region_dict.keys():
        
        # todo: add mechanism to check if table already exists and has rows
        # but not reaching the end date (most likely disconnected), and if yes continue on the last date scraped

        cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {key} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT,
                rank INTEGER,
                date TEXT,
                artist TEXT,
                url TEXT,
                region TEXT,
                streams INTEGER
            )
        ''')

        sub_chart = pd.DataFrame()

        for date in dates_list:
            date_string = date.strftime("%Y-%m-%d")
            driver.get("https://charts.spotify.com/charts/view/regional-" + key + "-daily/" + date_string)
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "tr")))
                print("Page load https://charts.spotify.com/charts/view/regional-" + key + "-daily/" + date_string + " success!")
                print("Scraping " + key + " " + date.strftime("%Y-%m-%d"))

                titles = driver.find_elements(By.CLASS_NAME, "kKOJRc")
                ranks = driver.find_elements(By.CLASS_NAME, "hgLxdb")
                artists = driver.find_elements(By.CLASS_NAME, "lfGOlT")
                artists = list(filter(lambda artists:\
                            len(artists.find_elements(By.CLASS_NAME, "bVVLJU")) > 0, artists))
                urls = driver.find_elements(By.CLASS_NAME, "gPJpnT")
                streams = driver.find_elements(By.CLASS_NAME, "cltvtH")

                for title, rank, artist_list, url, streams_count in zip (titles, ranks, artists, urls, streams):
                    title_string = title.text

                    rank_string = rank.text

                    artist_list_text_string = ""
                    artist_list_text = []
                    for artist in artist_list.find_elements(By.CLASS_NAME, "bVVLJU"):
                        artist_list_text.append(artist.text)
                    artist_list_text_string = ', '.join(map(str, artist_list_text))

                    anchor = url.find_element(By.TAG_NAME, "a")
                    anchor_string = anchor.get_attribute('href')

                    streams_count_string = streams_count.find_element(By.XPATH, './preceding-sibling::*[1]').text

                    cursor.execute(f'''INSERT INTO {key} (title, rank, date, artist, url, region, streams) VALUES (?, ?, ?, ?, ?, ?, ?)''',\
                                   (title_string, rank_string, date_string, artist_list_text_string, anchor_string, region_dict[key], streams_count_string))
                    # commit row to db
                    conn.commit()

                time.sleep(1)
            except:
                print("Page load https://charts.spotify.com/charts/view/regional-" + key + "-daily/" + date_string + " failed")
                print("Moving to next page..")
        
        print("sleeping for 10 seconds")
        print("***")
        time.sleep(10)
        
    conn.close()
    print("Last date reached, ending driver")
    driver.quit()
        
except:
    print("login failed, ending driver")
    conn.close()
    driver.quit()